# Preprocessing TCAT LoRaWAN transmission data

In [1]:
import os
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely as shp
import contextily as cx
import matplotlib.pyplot as plt
from shapely.geometry import box

In [2]:
# Set coorinate references
base = "EPSG:4326"  # Base system for lat, lon
web = "EPSG:3857"   # Web Mercator for plotting
utm = "EPSG:26918"  # Tompkins County, NY

In [5]:
# Load raw data from given directory
data_location  = "data/raw/"

data_files = os.listdir(data_location)

print(f"Loaded files in '{data_location}':")

for file in data_files:
    print(f"{file}")

Loaded files in 'data/raw/':
loc12112021_01_feather_crossbar.json
loc04112021_1_in_with_short_antenna.json
tcat_buses_1.json
loc03112021_1.json
loc04112021_2_in_with_longer_antenna.json
loc12112021_00_feather.json


In [6]:
# Read data to pandas DataFrame

data_frames = []

for file in data_files:
    df = pd.read_json(f"{data_location}{file}")
    df = df.drop(0)
    data_frames.append(df)
    
links = pd.concat(data_frames)

print(f"Loaded df with {links.shape[0]} rows and {links.shape[1]} columns.")

Loaded df with 1280 rows and 4 columns.


In [21]:
df = pd.read_csv('tcat-data-raw.csv')

In [25]:
df

,time,gps.busID,gps.groundSpeed,gps.latitude,gps.longitude,gps.motionHeading,gps.rx_metadata,gps.speedAccuracy,gps.timestamp
0,2022-06-09T18:43:05.357Z,1,7837,42.438965,-76.501640,173.869125,NaN,536,1654799044
1,2022-06-09T18:43:32.915Z,1,7837,42.438965,-76.501640,173.869125,NaN,536,1654799044
2,2022-06-09T19:33:03.703Z,1,369,42.439396,-76.494301,104.787209,NaN,371,1654803183
3,2022-06-09T20:23:00.752Z,1,8924,42.439129,-76.501770,176.304642,NaN,815,1654806180
4,2022-06-09T20:23:11.212Z,1,1984,42.438721,-76.501823,194.014786,NaN,868,1654806190
...,...,...,...,...,...,...,...,...,...
14575,2022-07-11T15:18:07.235Z,1,138,42.458073,-76.477165,339.131958,"[{""gateway_ids"":{""gateway_id"":""ttn-ithaca-00-0...",781,1657552686
14576,2022-07-11T15:18:28.236Z,1,1613,42.457939,-76.478355,260.489471,"[{""gateway_ids"":{""gateway_id"":""ttn-ithaca-00-0...",640,1657552707
14577,2022-07-11T15:21:48.730Z,1,11509,42.453476,-76.474083,227.755035,"[{""gateway_ids"":{""gateway_id"":""ttn-ithaca-00-0...",881,1657552908
14578,2022-07-11T15:22:42.736Z,1,3581,42.452591,-76.481018,252.706406,"[{""gateway_ids"":{""gateway_id"":""ttn-ithaca-00-0...",700,1657552962


In [24]:
df.query("gps.rx_metadata.notna()")

UndefinedVariableError: name 'gps' is not defined

## Process json columns

In [7]:
# Rename columns
links.rename(columns={"received_at": "received_at_meta"}, inplace=True)

# Normalize json columns
to_normalize = ["end_device_ids", "uplink_message"]

for col in to_normalize:
    temp = pd.json_normalize(links[col])
    links = links.join(temp)

links = links.drop(columns=to_normalize)
    
# Get uplink_id column
links = links.rename_axis('uplink_id').reset_index()

In [8]:
# Explode dataframe on "rx_metadata"
def to_list(x):
    if isinstance(x, dict):
        return [x]
    return x
    
links['rx_metadata'] = links['rx_metadata'].apply(to_list)
links = links.explode('rx_metadata')

# Get link_id column
links = links.reset_index().rename_axis('link_id').reset_index()
links = links.drop(columns='index')

In [9]:
# (json) Normalize rx_metadata
right = pd.json_normalize(links['rx_metadata'], max_level=1)
links = links.join(right)
links = links.drop(columns=['rx_metadata'])

In [10]:
# Rename columns
new_names = {'decoded_payload.altitude': 'altitude_payload',
             'decoded_payload.bat': 'bat_payload',
             'decoded_payload.hdop': 'hdop_payload',
             'decoded_payload.latitude': 'latitude_payload',
             'decoded_payload.longitude': 'longitude_payload',
             'settings.data_rate.lora.bandwidth': 'bandwidth',
             'settings.data_rate.lora.spreading_factor': 'spreading_factor',
             'settings.coding_rate': 'coding_rade',
             'settings.frequency': 'frequency',
             'settings.timestamp': 'timestamp',
             'settings.time': 'time',
             'locations.frm-payload.latitude':'latitude_frm_payload',
             'locations.frm-payload.longitude':'longitude_frm_payload',
             'locations.frm-payload.altitude': 'altitude_frm_payload',
             'gateway_ids.gateway_id': 'gateway_id',
             'gateway_ids.eui': 'gateway_eui',
             'location.latitude': 'latitude_rx',
             'location.longitude': 'longitude_rx',
             'location.altitude': 'altitude_rx',
            }

links = links.rename(columns=new_names)

In [11]:
# Check if coords match for various sources
cond1 = (links['latitude_payload'] == links['latitude_frm_payload'])
cond2 = (links['longitude_payload'] == links['longitude_frm_payload'])

n_obs = links.shape[0]
n_cnd = links[cond1 & cond2].shape[0]
print("'payload' and 'frm_payload' lat/lons match for %.2f%% of the observations." % (100*n_cnd/n_obs))

'payload' and 'frm_payload' lat/lons match for 97.38% of the observations.


In [12]:
# Clean tx geodata by using the best available source
for var in ['altitude', 'latitude', 'longitude']:
    
    # Use "*_frm_payload" when available, else "*_payload"
    links.loc[links[var+'_frm_payload'].isna(), var+'_frm_payload'] = links[var+'_payload']
    
    # Drop and rename
    links = links.drop(columns=[var+'_payload'])
    links = links.rename(columns={var+'_frm_payload': var+"_tx"})

In [13]:
# Retain only a subset of pretinent columns
retain_cols = ['link_id', 'uplink_id', 'device_id', 'dev_addr', 'received_at',
               'consumed_airtime', 'bandwidth', 'spreading_factor', 'coding_rade',
               'frequency', 'timestamp', 'latitude_tx', 'longitude_tx', 'altitude_tx',
               'timestamp', 'rssi', 'channel_rssi', 'snr', 'gateway_id', 'gateway_eui',
               'latitude_rx', 'longitude_rx', 'altitude_rx']

links = links[retain_cols].copy()

## Data cleaning

In [14]:
# Drop links with no rx_longitudes

nOld = links.shape[0]

links = links.drop(links.query("longitude_rx.isna()").index)
links = links.drop(links.query("altitude_rx.isna()").index)

nNew = links.shape[0]

print(f"Retained {nNew} rows, dropped {nOld - nNew}.")

Retained 1584 rows, dropped 94.


In [15]:
# Check whether all links have the necessary information
cols = ['latitude_rx',
        'longitude_rx',
        'altitude_rx',
        'latitude_tx',
        'longitude_tx',
        'altitude_tx']

n_total = links.shape[0]

for col in cols:
    if np.all(links[col].notna()):
        print(f"All rows have {col}-values")
    else:
        print(f"Some rows are missing {col}-values")

All rows have latitude_rx-values
All rows have longitude_rx-values
All rows have altitude_rx-values
All rows have latitude_tx-values
All rows have longitude_tx-values
Some rows are missing altitude_tx-values


## Expanding data set

The current data only consits of successfully transmitted links.

We are also interested in links that could have taken place but did not.

In [16]:
# uplink dataframe
uplinks = pd.DataFrame({'uplink_id':links['uplink_id'].unique()})

# get all GWs as list-like
gws = list(links['gateway_id'].unique())

# get gateway column that accepts lists
uplinks['gateway_id'] = [gws] * len(uplinks.index)

uplinks = uplinks.explode('gateway_id').reset_index(drop=True)

# Set multi-index
uplinks = uplinks.set_index(['uplink_id', 'gateway_id'])

# Join on right index
uplinks = uplinks.join(links.set_index(['uplink_id', 'gateway_id']), how='outer').reset_index()

In [17]:
# Set device data into large dataset
dev_cols = ['device_id', 'dev_addr', 'consumed_airtime',
            'bandwidth', 'spreading_factor', 'coding_rade',
            'frequency', 'latitude_tx', 'longitude_tx', 'altitude_tx' ]

for col in dev_cols:
    colmap = links.groupby('uplink_id')[col].first().to_dict()
    uplinks[col] = uplinks['uplink_id'].map(colmap)

In [18]:
# Set gateway data into large dataset
dev_cols = ['gateway_eui', 'latitude_rx',
            'longitude_rx', 'altitude_rx']

for col in dev_cols:
    colmap = links.groupby('gateway_id')[col].first().to_dict()
    uplinks[col] = uplinks['gateway_id'].map(colmap)

In [19]:
# Make success variable
uplinks['success'] = 0
uplinks.loc[uplinks['rssi'].notna(), 'success'] = 1

# Drop link id
uplinks = uplinks.drop(columns=['link_id'])

In [20]:
uplinks.shape

(13829, 25)

In [ ]:
# save data 
#uplinks.to_csv('ithaca_uplinks.csv', index=False)